In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.svm import SVR
from sklearn.kernel_ridge import KernelRidge
import math
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


/kaggle/input/playground-series-s3e14/sample_submission.csv
/kaggle/input/playground-series-s3e14/train.csv
/kaggle/input/playground-series-s3e14/test.csv


In [2]:
train_df = pd.read_csv('/kaggle/input/playground-series-s3e14/train.csv')
test_df = pd.read_csv('/kaggle/input/playground-series-s3e14/test.csv')

In [3]:
print(train_df.head())
train_y = train_df['yield']
train_df = train_df.drop('yield', axis='columns')

   id  clonesize  honeybee  bumbles  andrena  osmia  MaxOfUpperTRange  \
0   0       25.0      0.50     0.25     0.75   0.50              69.7   
1   1       25.0      0.50     0.25     0.50   0.50              69.7   
2   2       12.5      0.25     0.25     0.63   0.63              86.0   
3   3       12.5      0.25     0.25     0.63   0.50              77.4   
4   4       25.0      0.50     0.25     0.63   0.63              77.4   

   MinOfUpperTRange  AverageOfUpperTRange  MaxOfLowerTRange  MinOfLowerTRange  \
0              42.1                  58.2              50.2              24.3   
1              42.1                  58.2              50.2              24.3   
2              52.0                  71.9              62.0              30.0   
3              46.8                  64.7              55.8              27.0   
4              46.8                  64.7              55.8              27.0   

   AverageOfLowerTRange  RainingDays  AverageRainingDays  fruitset  fruitm

In [4]:
def normalize_column(df, column):
    df[column] = df[column]/df[column].max()
    return(df[column])


In [5]:
for column in train_df.columns:
    if(train_df[column].max() > 1) and (column != 'id'):
        train_df[column] = normalize_column(train_df, column)
        
for column in test_df.columns:
    if(test_df[column].max() > 1) and (column != 'id'):
        test_df[column] = normalize_column(test_df, column)

In [6]:
train_x = np.array(train_df.drop('id', axis='columns'))
test_x = np.array(test_df.drop('id', axis='columns'))
test_id = test_df['id']

In [7]:
X, X_test, y, y_test = train_test_split(train_x, train_y)
feature_selection = SelectKBest(f_classif, k=16)
X_new = feature_selection.fit_transform(X, y)
X_test_new = feature_selection.transform(X_test)
test_x_new = feature_selection.transform(test_x)


In [8]:
model = KernelRidge(alpha=.1)
model.fit(X_new, y)
train_pred = model.predict(X_new)
test_pred = model.predict(X_test_new)
print(math.sqrt(mean_squared_error(y, train_pred)))
print(math.sqrt(mean_squared_error(y_test, test_pred)))

582.4013660033957
595.9317886824306


In [9]:
results = model.predict(test_x_new)
results_df = pd.DataFrame({'id': test_id, 'yield': results})
results_df.to_csv('submission.csv', index=False)